# Scalar [Quantization](https://en.wikipedia.org/wiki/Quantization_(signal_processing)) of Digital Signals

In [ ]:
%matplotlib inline

import math
import numpy as np

import common

## A.  Uniform Quantization

### A.a. [Mid-tread ("round") Quantization](https://en.wikipedia.org/wiki/Quantization_(signal_processing)#Example)

Quantization index
\begin{equation}
k = \left\{ 
  \begin{array}{cl}
    \text{sign}(x) \left\lfloor \frac{\left| x \right|}{\Delta} + \frac{1}{2}\right\rfloor & \quad \text{if } x \leq 0 \\
    -\text{sign}(x) \left\lfloor \frac{\left| x \right|}{\Delta} + \frac{1}{2}\right\rfloor & \quad \textrm{if } x<0.
  \end{array}
\right.
\end{equation}

Reconstructed value
\begin{equation}
  y = \Delta k.
\end{equation}

The $k$ index can be also computed using the [round half toward zero](https://en.wikipedia.org/wiki/Rounding#Round_half_towards_zero) (or round to the nearest integer), for which NumPy provides the method [rint()](https://numpy.org/doc/stable/reference/generated/numpy.rint.html).

In [ ]:
!cat midtread_quantizer.py
import midtread_quantizer as midtread

In [ ]:
quantization_step = 2

In [ ]:
x = np.arange(11)-5

In [ ]:
print("x =", x)
print("k =", np.rint(x/quantization_step).astype(np.int16))

### A.b. [Mid-rise ("truncation") Quantization](https://en.wikipedia.org/wiki/Quantization_(signal_processing)#Mid-riser_and_mid-tread_uniform_quantizers)

Quantization index
\begin{equation}
  k=\Big\lfloor \frac{x}{\Delta}\Big\rfloor.
\end{equation}

Reconstructed value
\begin{equation}
  y = \Delta \Big(k + \frac{1}{2}\Big).
\end{equation}

In [ ]:
!cat midrise_quantizer.py
import midrise_quantizer as midrise

In [ ]:
print("x =", x)
print("k =", np.floor(x/quantization_step).astype(np.int16))

### A.c. [Dead-zone Quantization](https://en.wikipedia.org/wiki/Quantization_(signal_processing)#Dead-zone_quantizers)

See *JPEG2000 Image Compression Fundamentals, Standards and Practice*.

Quantization index
\begin{equation}
k = \left\{ 
  \begin{array}{cl}
    \text{sign}(x) \left\lfloor \frac{\left| x \right|}{\Delta}\right\rfloor & \quad \text{if } \frac{\left| x \right|}{\Delta} >0 \\
    0                 & \quad \textrm{otherwise},
  \end{array}
\right.
\end{equation}
which can be computed efficiently in NumPy by simply converting the floating point representation of $x/\Delta$ to an integer using the [astype()](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.astype.html) method.
Reconstructed value

\begin{equation}
y = \left\{ 
  \begin{array}{cl}
    0                                          & \quad \text{if } k=0 \\
    \text{sign}(k) (\left| k \right|-0.5)\Delta & \quad \text{if } k\ne 0 
  \end{array}
\right.
\end{equation}

Using the same simplification, the second equation boils down to
\begin{equation}
  y = \Delta k.
\end{equation}


In [ ]:
!cat deadzone_quantizer.py
import deadzone_quantizer as deadzone

In [ ]:
print("x =", x)
print("k =", np.sign(x)*np.floor(np.abs(x)/quantization_step).astype(np.int16))
print("k =", (x/quantization_step).astype(np.int16))

## B. [Non Uniform Quantization](https://nptel.ac.in/content/storage2/courses/117104069/chapter_5/5_5.html)

### B.a. [Companded (COMpressed + exPANDED)](https://en.wikipedia.org/wiki/Companding) Quantization

#### B.a.1. [$\mu$-Law](https://en.wikipedia.org/wiki/%CE%9C-law_algorithm) Companded  Quantization

Compressor
\begin{equation}
C(x) = \text{sgn}(x) \frac{\ln(1+ \mu |x|)}{\ln(1+\mu)}, ~~~~-1 \leq x \leq 1,
\end{equation}
shere $\mu=255$ in most implementations.

Expander:
\begin{equation}
C^{-1}(y) = \text{sgn}(y) (1 / \mu ) ((1 + \mu)^{|y|}- 1),~~~~-1 \leq y \leq 1.
\end{equation}

In [ ]:
!cat companded_quantizer.py
import companded_quantizer as companded

In [ ]:
x = np.linspace(-1, 1, 500)

mu = 255
y = companded.muLaw_compress(x, mu)
common.plot(x, y, "Input", "Output", "$\mu$-law Compressor ($\mu={}$)".format(mu))

In [ ]:
x = np.linspace(-1, 1, 500)

mu = 255
y = companded.muLaw_expand(x, mu)
common.plot(x, y, "Input", "Output", "$\mu$-law Expander ($\mu={}$)".format(mu))

In [ ]:
mu = 255
x = np.linspace(-1, 1, 500)
y = companded.muLaw_compress(x, mu)
x_recons = companded.muLaw_expand(y, mu)
common.plot(x, x_recons, "Input", "Output", "Expansion(Compression(Input))".format(mu))

After these definitions, we define the quantization index
\begin{equation}
  k = Q\big(C(x)\big),
\end{equation}
where $C$ is the compression function and $Q$ is a dead-zone quantizer. 

Reconstruction value
\begin{equation}
  y = C^{-1}\big(Q^{-1}(k)\big),
\end{equation}
where $Q^{-1}$ stands for the dead-zone dequantizer and $C^{-1}$ for the expander function.

## Comparing Quantizers I/O

In [ ]:
quantization_step = 1 # Delta
x = np.linspace(-8, 8, 500) # Input samples
k_T = midtread.quantize(x, quantization_step) # Quantized samples
y_T = midtread.dequantize(k_T, quantization_step) # Reconstructed samples
k_R = midrise.quantize(x, quantization_step)
y_R = midrise.dequantize(k_R, quantization_step)
k_D = deadzone.quantize(x, quantization_step)
y_D = deadzone.dequantize(k_D, quantization_step)
k_C = companded.quantize(x, quantization_step)
y_C = companded.dequantize(k_C, quantization_step)
common.plot(x, y_T, "Input Sample", "Reconstructed Sample", "Mid-tread Quantizer ($\Delta={}$)".format(quantization_step))
common.plot(x, y_R, "Input Sample", "Reconstructed Sample", "Mid-rise Quantizer ($\Delta={}$)".format(quantization_step))
common.plot(x, y_D, "Input Sample", "Reconstructed Sample", "Dead-zone Quantizer ($\Delta={}$)".format(quantization_step))
common.plot(x, y_C, "Input Sample", "Reconstructed Sample", "Companded Dead-zone $\mu$-Law Quantizer ($\mu={}, \Delta={}$)".format(mu, quantization_step))

In [ ]:
quantization_step = 2 # Delta
x = np.linspace(-8, 8, 500) # Input samples
k_T = midtread.quantize(x, quantization_step) # Quantized samples
y_T = midtread.dequantize(k_T, quantization_step) # Reconstructed samples
k_R = midrise.quantize(x, quantization_step)
y_R = midrise.dequantize(k_R, quantization_step)
k_D = deadzone.quantize(x, quantization_step)
y_D = deadzone.dequantize(k_D, quantization_step)
k_C = companded.quantize(x, quantization_step)
y_C = companded.dequantize(k_C, quantization_step)
common.plot(x, y_T, "Input Sample", "Reconstructed Sample", "Mid-tread Quantizer ($\Delta={}$)".format(quantization_step))
common.plot(x, y_R, "Input Sample", "Reconstructed Sample", "Mid-rise Quantizer ($\Delta={}$)".format(quantization_step))
common.plot(x, y_D, "Input Sample", "Reconstructed Sample", "Dead-zone Quantizer ($\Delta={}$)".format(quantization_step))
common.plot(x, y_C, "Input Sample", "Reconstructed Sample", "Companded Dead-zone $\mu$-Law Quantizer ($\mu={}, \Delta={}$)".format(mu, quantization_step))

In [ ]:
quantization_step = 3 # Delta
x = np.linspace(-8, 8, 500) # Input samples
k_T = midtread.quantize(x, quantization_step) # Quantized samples
y_T = midtread.dequantize(k_T, quantization_step) # Reconstructed samples
k_R = midrise.quantize(x, quantization_step)
y_R = midrise.dequantize(k_R, quantization_step)
k_D = deadzone.quantize(x, quantization_step)
y_D = deadzone.dequantize(k_D, quantization_step)
k_C = companded.quantize(x, quantization_step)
y_C = companded.dequantize(k_C, quantization_step)
common.plot(x, y_T, "Input Sample", "Reconstructed Sample", "Mid-tread Quantizer ($\Delta={}$)".format(quantization_step))
common.plot(x, y_R, "Input Sample", "Reconstructed Sample", "Mid-rise Quantizer ($\Delta={}$)".format(quantization_step))
common.plot(x, y_D, "Input Sample", "Reconstructed Sample", "Deadzone Quantizer ($\Delta={}$)".format(quantization_step))
common.plot(x, y_C, "Input Sample", "Reconstructed Sample", "Companded Dead-zone $\mu$-Law Quantizer ($\mu={}, \Delta={}$)".format(mu, quantization_step))

In [ ]:
quantization_step = 4 # Delta
x = np.linspace(-8, 8, 500) # Input samples
k_T = midtread.quantize(x, quantization_step) # Quantized samples
y_T = midtread.dequantize(k_T, quantization_step) # Reconstructed samples
k_R = midrise.quantize(x, quantization_step)
y_R = midrise.dequantize(k_R, quantization_step)
k_D = deadzone.quantize(x, quantization_step)
y_D = deadzone.dequantize(k_D, quantization_step)
k_C = companded.quantize(x, quantization_step)
y_C = companded.dequantize(k_C, quantization_step)
common.plot(x, y_T, "Input Sample", "Reconstructed Sample", "Mid-tread Quantizer ($\Delta={}$)".format(quantization_step))
common.plot(x, y_R, "Input Sample", "Reconstructed Sample", "Mid-rise Quantizer ($\Delta={}$)".format(quantization_step))
common.plot(x, y_D, "Input Sample", "Reconstructed Sample", "Dead-zone Quantizer ($\Delta={}$)".format(quantization_step))
common.plot(x, y_C, "Input Sample", "Reconstructed Sample", "Companded Dead-zone $\mu$-Law Quantizer ($\mu={}, \Delta={}$)".format(mu, quantization_step))

In [ ]:
quantization_step = 1024
x = np.linspace(-32768, 32767, 65536)
k_T = midtread.quantize(x, quantization_step) # Quantized samples
y_T = midtread.dequantize(k_T, quantization_step) # Reconstructed samples
k_R = midrise.quantize(x, quantization_step)
y_R = midrise.dequantize(k_R, quantization_step)
k_D = deadzone.quantize(x, quantization_step)
y_D = deadzone.dequantize(k_D, quantization_step)
k_C = companded.quantize(x, quantization_step)
y_C = companded.dequantize(k_C, quantization_step)
common.plot(x, y_T, "Input Sample", "Reconstructed Sample", "Mid-tread Quantizer ($\Delta={}$)".format(quantization_step))
common.plot(x, y_R, "Input Sample", "Reconstructed Sample", "Mid-rise Quantizer ($\Delta={}$)".format(quantization_step))
common.plot(x, y_D, "Input Sample", "Reconstructed Sample", "Dead-zone Quantizer ($\Delta={}$)".format(quantization_step))
common.plot(x, y_C, "Input Sample", "Reconstructed Sample", "Companded Dead-zone $\mu$-Law Quantizer ($\mu={}, \Delta={}$)".format(mu, quantization_step))

## Comparing quantization error

In [ ]:
quantization_step = 1 # Delta
x = np.linspace(-8, 8, 500) # Input samples
k_T = midtread.quantize(x, quantization_step) # Quantized samples
y_T = midtread.dequantize(k_T, quantization_step) # Reconstructed samples
k_R = midrise.quantize(x, quantization_step)
y_R = midrise.dequantize(k_R, quantization_step)
k_D = deadzone.quantize(x, quantization_step)
y_D = deadzone.dequantize(k_D, quantization_step)
k_C = companded.quantize(x, quantization_step)
y_C = companded.dequantize(k_C, quantization_step)
error_T = x - y_T
error_R = x - y_R
error_D = x - y_D
error_C = x - y_C
common.plot(x, error_T, "Input Sample", "Quantization Error", "Mid-tread Quantizer ($\Delta={}$)".format(quantization_step))
common.plot(x, error_R, "Input Sample", "Quantization Error", "Mid-rise Quantizer ($\Delta={}$)".format(quantization_step))
common.plot(x, error_D, "Input Sample", "Quantization Error", "Dead-zone Quantizer ($\Delta={}$)".format(quantization_step))
common.plot(x, error_C, "Input Sample", "Quantization Error", "Companded Dead-zone $\mu$-Law Quantizer ($\mu={}, \Delta={}$)".format(mu, quantization_step))

In [ ]:
quantization_step = 2 # Delta
x = np.linspace(-8, 8, 500) # Input samples
k_T = midtread.quantize(x, quantization_step) # Quantized samples
y_T = midtread.dequantize(k_T, quantization_step) # Reconstructed samples
k_R = midrise.quantize(x, quantization_step)
y_R = midrise.dequantize(k_R, quantization_step)
k_D = deadzone.quantize(x, quantization_step)
y_D = deadzone.dequantize(k_D, quantization_step)
k_C = companded.quantize(x, quantization_step)
y_C = companded.dequantize(k_C, quantization_step)
error_T = x - y_T
error_R = x - y_R
error_D = x - y_D
error_C = x - y_C
common.plot(x, error_T, "Input Sample", "Quantization Error", "Mid-tread Quantizer ($\Delta={}$)".format(quantization_step))
common.plot(x, error_R, "Input Sample", "Quantization Error", "Mid-rise Quantizer ($\Delta={}$)".format(quantization_step))
common.plot(x, error_D, "Input Sample", "Quantization Error", "Dead-zone Quantizer ($\Delta={}$)".format(quantization_step))
common.plot(x, error_C, "Input Sample", "Quantization Error", "Companded Dead-zone $\mu$-Law Quantizer ($\mu={}, \Delta={}$)".format(mu, quantization_step))

In [ ]:
quantization_step = 1024 # Delta
x = np.linspace(-32768, 32767, 500) # Input samples
k_T = midtread.quantize(x, quantization_step) # Quantized samples
y_T = midtread.dequantize(k_T, quantization_step) # Reconstructed samples
k_R = midrise.quantize(x, quantization_step)
y_R = midrise.dequantize(k_R, quantization_step)
k_D = deadzone.quantize(x, quantization_step)
y_D = deadzone.dequantize(k_D, quantization_step)
k_C = companded.quantize(x, quantization_step)
y_C = companded.dequantize(k_C, quantization_step)
error_T = x - y_T
error_R = x - y_R
error_D = x - y_D
error_C = x - y_C
common.plot(x, error_T, "Input Sample", "Quantization Error", "Mid-tread Quantizer ($\Delta={}$)".format(quantization_step))
common.plot(x, error_R, "Input Sample", "Quantization Error", "Mid-rise Quantizer ($\Delta={}$)".format(quantization_step))
common.plot(x, error_D, "Input Sample", "Quantization Error", "Dead-zone Quantizer ($\Delta={}$)".format(quantization_step))
common.plot(x, error_C, "Input Sample", "Quantization Error", "Companded Dead-zone $\mu$-Law Quantizer ($\mu={}, \Delta={}$)".format(mu, quantization_step))

## Working with signed integers of 16 bits

In [ ]:
quantization_step = 1
x = np.linspace(-32768, 32767, 65536).astype(np.int16)
k_T = midtread.quantize(x, quantization_step)
y_T = midtread.dequantize(k_T, quantization_step)
k_R = midrise.quantize(x, quantization_step)
y_R = midrise.dequantize(k_R, quantization_step)
k_D = deadzone.quantize(x, quantization_step)
y_D = deadzone.dequantize(k_D, quantization_step)
k_C = companded.quantize(x, quantization_step)
y_C = companded.dequantize(k_C, quantization_step)

n = 16
print(f"{'Mid-tread':>20s} {'Mid-rise':>20s} {'Dead-zone':>20s} {'Companded Dead-zone':>20s}")
print(f"{'Input':>6s} {'Output':>6s} {'Recons':>6s} {'Input':>6s} {'Output':>6s} {'Recons':>6s} {'Input':>6s} {'Output':>6s} {'Recons':>6s} {'Input':>6s} {'Output':>6s} {'Recons':>6s}")
offset = (len(x)-n)//2
for i in range(n):
    input = int(x[i+offset])
    output_T = int(k_T[i+offset])
    recons_T = int(y_T[i+offset])
    print(f"{input:>6d} {output_T:>6d} {recons_T:>6d}", end='')
    output_R = int(k_R[i+offset])
    recons_R = int(y_R[i+offset])
    print(f" {input:>6d} {output_R:>6d} {recons_R:>6d}", end='')
    output_D = int(k_D[i+offset])
    recons_D = int(y_D[i+offset])
    print(f" {input:>6d} {output_D:>6d} {recons_D:>6d}", end='')
    output_C = int(k_C[i+offset])
    recons_C = int(y_C[i+offset])
    print(f" {input:>6d} {output_C:>6d} {recons_C:>6d}")


In [ ]:
quantization_step = 2
x = np.linspace(-32768, 32767, 65536).astype(np.int16)
k_T = midtread.quantize(x, quantization_step)
y_T = midtread.dequantize(k_T, quantization_step)
k_R = midrise.quantize(x, quantization_step)
y_R = midrise.dequantize(k_R, quantization_step)
k_D = deadzone.quantize(x, quantization_step)
y_D = deadzone.dequantize(k_D, quantization_step)
k_C = companded.quantize(x, quantization_step)
y_C = companded.dequantize(k_C, quantization_step)

n = 16
print(f"{'Mid-tread':>20s} {'Mid-rise':>20s} {'Dead-zone':>20s} {'Companded Dead-zone':>20s}")
print(f"{'Input':>6s} {'Output':>6s} {'Recons':>6s} {'Input':>6s} {'Output':>6s} {'Recons':>6s} {'Input':>6s} {'Output':>6s} {'Recons':>6s} {'Input':>6s} {'Output':>6s} {'Recons':>6s}")
offset = (len(x)-n)//2
for i in range(n):
    input = int(x[i+offset])
    output_T = int(k_T[i+offset])
    recons_T = int(y_T[i+offset])
    print(f"{input:>6d} {output_T:>6d} {recons_T:>6d}", end='')
    output_R = int(k_R[i+offset])
    recons_R = int(y_R[i+offset])
    print(f" {input:>6d} {output_R:>6d} {recons_R:>6d}", end='')
    output_D = int(k_D[i+offset])
    recons_D = int(y_D[i+offset])
    print(f" {input:>6d} {output_D:>6d} {recons_D:>6d}", end='')
    output_C = int(k_C[i+offset])
    recons_C = int(y_C[i+offset])
    print(f" {input:>6d} {output_C:>6d} {recons_C:>6d}")


In [ ]:
quantization_step = 32
x = np.linspace(-32768, 32767, 65536).astype(np.int16)
k_T = midtread.quantize(x, quantization_step)
y_T = midtread.dequantize(k_T, quantization_step)
k_R = midrise.quantize(x, quantization_step)
y_R = midrise.dequantize(k_R, quantization_step)
k_D = deadzone.quantize(x, quantization_step)
y_D = deadzone.dequantize(k_D, quantization_step)
k_C = companded.quantize(x, quantization_step)
y_C = companded.dequantize(k_C, quantization_step)

n = 16
print(f"{'Mid-tread':>20s} {'Mid-rise':>20s} {'Dead-zone':>20s} {'Companded Dead-zone':>20s}")
print(f"{'Input':>6s} {'Output':>6s} {'Recons':>6s} {'Input':>6s} {'Output':>6s} {'Recons':>6s} {'Input':>6s} {'Output':>6s} {'Recons':>6s} {'Input':>6s} {'Output':>6s} {'Recons':>6s}")
offset = (len(x)-n)//2
for i in range(n):
    input = int(x[i+offset])
    output_T = int(k_T[i+offset])
    recons_T = int(y_T[i+offset])
    print(f"{input:>6d} {output_T:>6d} {recons_T:>6d}", end='')
    output_R = int(k_R[i+offset])
    recons_R = int(y_R[i+offset])
    print(f" {input:>6d} {output_R:>6d} {recons_R:>6d}", end='')
    output_D = int(k_D[i+offset])
    recons_D = int(y_D[i+offset])
    print(f" {input:>6d} {output_D:>6d} {recons_D:>6d}", end='')
    output_C = int(k_C[i+offset])
    recons_C = int(y_C[i+offset])
    print(f" {input:>6d} {output_C:>6d} {recons_C:>6d}")
